<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/TextCNN_with_vocab_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/6b20ab3c-04a2-4b5b-b39b-6531835e3571

In [ ]:
pip install tensorflow

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
file_path = './saved_file'
file_path = ('posts.xlsx')
data = pd.read_excel(file_path)

# Preprocess the data

In [5]:
# Preprocess the data
X = data['Question_body']
y = data['Label']

In [6]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)


In [9]:
# Parameters
vocab_size = 5000
embedding_dim = 100
max_length = 500
trunc_type = 'post'
padding_type = 'post'


In [10]:
# Tokenize the data
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [11]:
# Pad the sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)


# TextCNN model
https://chatgpt.com/share/d5dd93d5-d7a7-4488-9bfb-8824d7cffe39

The create_text_cnn_model function you provided constructs a TextCNN model for binary text classification. This model uses multiple convolutional layers with different kernel sizes to capture various features from the text data, followed by pooling and concatenation. Here's a detailed explanation of the components:

Inputs Layer: Specifies the input shape, which is the maximum length of the sequences.
Embedding Layer: Transforms input tokens into dense vectors of fixed size (embedding_dim).
Convolutional and Pooling Layers: Three sets of convolutional layers with different kernel sizes (3, 4, and 5) followed by max pooling. These layers help in capturing different n-gram features from the text.
Concatenate Layer: Concatenates the outputs of the pooling layers along the specified axis.
Flatten Layer: Flattens the concatenated outputs into a single dimension.
Dense Layer: A fully connected layer with ReLU activation.
Dropout Layer: Helps prevent overfitting by randomly dropping units during training.
Output Layer: A single neuron with sigmoid activation for binary classification.
Compile Model: Configures the model for training with binary cross-entropy loss, the Adam optimizer, and accuracy as a metric.

In [12]:
# TextCNN model
def create_text_cnn_model(vocab_size, embedding_dim, max_length):
    inputs = Input(shape=(max_length,))
    embedding = Embedding(vocab_size, embedding_dim, input_length=max_length)(inputs)

    conv1 = Conv1D(128, 3, activation='relu')(embedding)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(128, 4, activation='relu')(embedding)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(128, 5, activation='relu')(embedding)
    pool3 = MaxPooling1D(pool_size=2)(conv3)

    concatenated = concatenate([pool1, pool2, pool3], axis=1)
    flatten = Flatten()(concatenated)
    dense1 = Dense(128, activation='relu')(flatten)
    dropout = Dropout(0.5)(dense1)
    outputs = Dense(1, activation='sigmoid')(dropout)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create the model

In [13]:
# Create the model
text_cnn_model = create_text_cnn_model(vocab_size, embedding_dim, max_length)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# Display the model summary

In [14]:
# Display the model summary
text_cnn_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 500)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 500, 100)       │        500,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 498, 128)       │         38,528 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 497, 128)       │         51,328 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 496, 128)       │         64,128 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 249, 128)       │              0 │ conv1d[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 248, 128)       │              0 │ conv1d_1[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_2           │ (None, 248, 128)       │              0 │ conv1d_2[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 745, 128)       │              0 │ max_pooling1d[0][0],   │
│                           │                        │                │ max_pooling1d_1[0][0], │
│                           │                        │                │ max_pooling1d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 95360)          │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     12,206,208 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 12,860,321 (49.06 MB)

 Trainable params: 12,860,321 (49.06 MB)

 Non-trainable params: 0 (0.00 B)

# Train the model

In [15]:
# Train the model
history = text_cnn_model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = text_cnn_model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.8667 - loss: 0.2872 - val_accuracy: 0.9674 - val_loss: 0.1036
Epoch 2/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9786 - loss: 0.0694 - val_accuracy: 0.9688 - val_loss: 0.1097
Epoch 3/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9837 - loss: 0.0480 - val_accuracy: 0.9699 - val_loss: 0.1653
Epoch 4/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9920 - loss: 0.0250 - val_accuracy: 0.9656 - val_loss: 0.2030
Epoch 5/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9957 - loss: 0.0126 - val_accuracy: 0.9661 - val_loss: 0.2402
Epoch 6/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9980 - loss: 0.0071 - val_accuracy: 0.9650 - val_loss: 0.3584
Epoch 7/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9970 - loss: 0.0140 - val_accuracy: 0.9654 - val_loss: 0.3363
Epoch 8/10
327/327 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9980 - loss: 0.0072 - val_

# Predict on new data

In [22]:
# Predict on new data
y_pred = text_cnn_model.predict(X_test_padded)
y_pred_classes = (y_pred > 0.5).astype("int32")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


# Evaluate predictions

In [23]:
# Evaluate predictions
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred_classes)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2265
           1       0.96      0.96      0.96      2215

    accuracy                           0.96      4480
   macro avg       0.96      0.96      0.96      4480
weighted avg       0.96      0.96      0.96      4480



In [24]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# Compute metrics for each class
precision_class, recall_class, f1_class, support_class = precision_recall_fscore_support(y_pred_classes, y_pred, average=None, labels=[0, 1])

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred, labels=[0, 1])

# Extract TP, TN, FP, FN for each class
TP_0 = conf_matrix[0, 0]
FN_0 = conf_matrix[0, 1]
FP_0 = conf_matrix[1, 0]
TN_0 = conf_matrix[1, 1]

TP_1 = conf_matrix[1, 1]
FN_1 = conf_matrix[1, 0]
FP_1 = conf_matrix[0, 1]
TN_1 = conf_matrix[0, 0]

# Calculate accuracy for each class
accuracy_class_0 = (TP_0 + TN_0) / (TP_0 + TN_0 + FP_0 + FN_0)
accuracy_class_1 = (TP_1 + TN_1) / (TP_1 + TN_1 + FP_1 + FN_1)

print(f'Class 0 - Precision: {precision_class[0]}, Recall: {recall_class[0]}, Accuracy: {accuracy_class_0}, F1-score: {f1_class[0]}, Support: {support_class[0]}')
print(f'Class 1 - Precision: {precision_class[1]}, Recall: {recall_class[1]}, Accuracy: {accuracy_class_1}, F1-score: {f1_class[1]}, Support: {support_class[1]}')


ValueError: Classification metrics can't handle a mix of binary and continuous targets